In [ ]:
%pip -q install google-genai

In [2]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [4]:
# Instalar Framework de agentes do Google ################################################
!pip install -q google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 26.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72

In [5]:
from google import genai
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search  # Importante para permitir buscas
from google.genai import types
from IPython.display import display, Markdown
import textwrap
import warnings

warnings.filterwarnings("ignore")

client = genai.Client()
MODEL_ID = "gemini-2.0-flash"

In [ ]:
def call_agent(agent: Agent, message_text: str) -> str:
    session_service = InMemorySessionService()
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
            for part in event.content.parts:
                if part.text is not None:
                    final_response += part.text
                    final_response += "\n"
    return final_response


def to_markdown(text):
    text = text.replace('•', '  *')
    return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))


####################################################
# --- Agente de Adoção: Orientações e Formulário --- #
####################################################

adocao_agent = Agent(
    name="agente_adocao",
    model=MODEL_ID,  # Use o mesmo MODEL_ID definido anteriormente
    instruction="""
    Você é um assistente de adoção de animais. Sua tarefa é ajudar pessoas interessadas
    no processo de adoção, fornecendo informações, respondendo perguntas e guiando-as
    para o preenchimento de um formulário (link fictício fornecido para demonstração).

    Siga estas etapas:

    1. Cumprimente o usuário e pergunte como pode ajudar.
    2. Responda a perguntas sobre adoção, como requisitos, cuidados com os animais, etc.
            Use a ferramenta google_search para buscar informações adicionais, se necessário.
    3. Ofereça o link para o formulário de adoção: [https://docs.google.com/forms/d/e/1FAIpQLSe7hDZPL59al_Gd1369i6p6Yx736SHhexiCl-NtMB9oYQHyTw/viewform?usp=header].
    4. Se o usuário tiver dúvidas sobre o formulário, ajude-o a preenchê-lo.
    5. Agradeça ao usuário e finalize a conversa.

    Seja amigável, prestativo e encoraje a adoção responsável.
    """,
    description="Agente para auxiliar no processo de adoção de animais.",
    tools=[google_search]  # Adicione a ferramenta google_search
)



# --- Loop principal aprimorado ---

print("🐾 Agente de Adoção de Animais 🐾")
print("Oi! Eu posso te ajudar com adoção de animais. Pode perguntar sobre os animais disponíveis, cuidados, ou pedir o formulário 🐶🐱")
planilha_animais_disponiveis = "https://docs.google.com/spreadsheets/d/1U3EBw1YZR0Qt_iqeWZD0rb2kpbogJENbyIe72XXRXws/edit?usp=sharing"



while True:
    user_input = input("❓ Como posso ajudar? (Digite 'sair' para encerrar): ").lower().strip()

    if user_input in ["sair", "encerrar", "tchau"]:
        print("Foi um prazer te ajudar! Obrigado por considerar a adoção responsável 💚🐾")
        break

    elif "adotar" in user_input and "gato" in user_input:
        print("\n😺 Que ótimo que quer adotar um gatinho! Você pode ver alguns dos animais disponíveis aqui:")
        print("📋 Planilha com animais para adoção: https://docs.google.com/spreadsheets/d/1U3EBw1YZR0Qt_iqeWZD0rb2kpbogJENbyIe72XXRXws/edit?usp=sharing")
        print("Se gostar de algum, preencha este formulário: https://docs.google.com/forms/d/e/1FAIpQLSe7hDZPL59al_Gd1369i6p6Yx736SHhexiCl-NtMB9oYQHyTw/viewform?usp=header")

    elif "formulário" in user_input or "formulario" in user_input:
        print("\n📝 Você pode preencher o formulário por este link:")
        print("👉 https://docs.google.com/forms/d/e/1FAIpQLSe7hDZPL59al_Gd1369i6p6Yx736SHhexiCl-NtMB9oYQHyTw/viewform?usp=header")
        print("Se tiver dúvidas sobre o que preencher, é só me perguntar!")

    elif "castração" in user_input or "castrado" in user_input:
        print("\n📌 Todos os nossos animais são castrados antes da adoção. Isso é fundamental para o bem-estar deles e para evitar abandonos futuros.")
        print("Se tiver dúvidas sobre o procedimento ou benefícios, posso te explicar mais!")

    elif "vacina" in user_input:
        print("\n💉 Nossos animais são entregues com a vacinação em dia, conforme a idade. Caso precise saber sobre reforços ou vacinas obrigatórias, posso te ajudar!")

    elif "adoção responsável" in user_input or "responsável" in user_input:
        print("\n🐾 Adoção responsável significa garantir que o animal tenha um lar seguro, com alimentação, atenção, cuidados veterinários e muito amor.")
        print("Se quiser, posso te enviar uma checklist para avaliar se está tudo pronto pra receber um novo amigo!")

    else:
        # fallback: deixar o agente Gemini responder
        response = call_agent(adocao_agent, user_input)
        display(to_markdown(response))

    print("--------------------------------------------------------------")


🐾 Agente de Adoção de Animais 🐾
Oi! Eu posso te ajudar com adoção de animais. Pode perguntar sobre os animais disponíveis, cuidados, ou pedir o formulário 🐶🐱
❓ Como posso ajudar? (Digite 'sair' para encerrar): olá, estou buscando um cão para adotar


> Olá! Que ótimo que você está pensando em adotar um cão! É uma decisão maravilhosa e tenho certeza de que você e seu futuro amigo peludo serão muito felizes juntos. 😊
> 
> Como posso te ajudar nesse processo de adoção? Você tem alguma dúvida sobre os requisitos para adotar, os cuidados que um cão precisa, ou algo mais?
> 


--------------------------------------------------------------
❓ Como posso ajudar? (Digite 'sair' para encerrar): queria ver os animais disponiveis


> Olá! Tudo bem? Sou seu agente de adoção e estou aqui para te ajudar a encontrar o companheiro perfeito! 😊
> 
> No momento, não consigo mostrar os animais disponíveis diretamente por aqui. Mas posso te ajudar com informações sobre o processo de adoção, os requisitos e os cuidados necessários para receber um novo amigo em sua casa.
> 
> Para começar, você já tem alguma preferência por tipo de animal (cão, gato, etc.) ou alguma outra característica específica que esteja buscando? Assim, posso te dar dicas mais direcionadas e te ajudar a se preparar para a adoção!
> 
> Se quiser, posso te enviar o link do nosso formulário de adoção para você dar uma olhada. Lá você encontrará mais informações e poderá se inscrever para conhecer nossos animais disponíveis! 😊
> 


--------------------------------------------------------------
